In [ ]:
# import os, sys, time, threading
from datetime import datetime
from collections import deque
from threading import Lock, Event
import pandas as pd
import numpy as np
from dash import Dash, dcc, html, Output, Input
import plotly.graph_objects as go
from arduino_iot_cloud import ArduinoCloudClient
from iot_secrets.secrets import DEVICE_ID, SECRET_KEY

# Parameters
N_SAMPLES = 150         # batch size for saving CSV
PLOT_WINDOW = 1000      # max points in live graph
OUT_DATA_DIR = "week8_data"
os.makedirs(OUT_DATA_DIR, exist_ok=True)

# Variable names to watch in Cloud
X_NAMES = ["x", "py_x", "accelerometer_X", "Accelerometer_X", "accel_x"]
Y_NAMES = ["y", "py_y", "accelerometer_Y", "Accelerometer_Y", "accel_y"]
Z_NAMES = ["z", "py_z", "accelerometer_Z", "Accelerometer_Z", "accel_z"]

# State
lock = Lock()
stop_event = Event()
latest = {"x": None, "y": None, "z": None}
ingest = deque()
history = deque(maxlen=20000)
_last_fig = None

def ts_file():
    return datetime.now().strftime("%Y%m%d-%H%M%S")

def make_fig(df):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df["timestamp"], y=df["x"], name="x"))
    fig.add_trace(go.Scatter(x=df["timestamp"], y=df["y"], name="y"))
    fig.add_trace(go.Scatter(x=df["timestamp"], y=df["z"], name="z"))
    fig.update_layout(
        margin=dict(l=30, r=10, t=30, b=30),
        xaxis_title="Time", yaxis_title="Accel"
    )
    return fig

def summarize(df):
    s = {
        "x_mean": df["x"].mean(), "x_std": df["x"].std(),
        "y_mean": df["y"].mean(), "y_std": df["y"].std(),
        "z_mean": df["z"].mean(), "z_std": df["z"].std(),
    }
    return "\n".join(f"{k:8s}: {v: .5f}" for k, v in s.items())

def flush_remaining(tag="partial"):
    with lock:
        if not ingest:
            print("No leftover samples to save.")
            return
        rows = list(ingest); ingest.clear()
    df = pd.DataFrame(rows, columns=["timestamp","x","y","z"])
    base = ts_file()
    csv_path = os.path.join(OUT_DATA_DIR, f"accel_batch_{tag}_{base}.csv")
    df_out = df.copy()
    df_out["timestamp"] = [t.strftime("%Y-%m-%d %H:%M:%S") for t in df_out["timestamp"]]
    df_out.to_csv(csv_path, index=False)
    print("Flushed remaining to:", csv_path)

def append_sample(axis, value):
    with lock:
        latest[axis] = value
        if all(v is not None for v in latest.values()):
            row = (datetime.now(), latest["x"], latest["y"], latest["z"])
            ingest.append(row)
            history.append(row)

# Cloud callbacks
def on_x(client, value): append_sample("x", value)
def on_y(client, value): append_sample("y", value)
def on_z(client, value): append_sample("z", value)

def cloud_loop():
    while not stop_event.is_set():
        try:
            client = ArduinoCloudClient(
                device_id=DEVICE_ID,
                username=DEVICE_ID,
                password=SECRET_KEY,
                sync_mode=True
            )
            def reg(names, cb):
                for n in names: client.register(n, value=None, on_write=cb)
            reg(X_NAMES, on_x); reg(Y_NAMES, on_y); reg(Z_NAMES, on_z)
            print("Connecting to Arduino IoT Cloud…")
            client.start()
            print("Connected. Press 'q' to stop capture.")
            while not stop_event.is_set():
                client.update()
                time.sleep(0.05)
        except Exception as e:
            print("Disconnected, retrying in 3s:", e)
            time.sleep(3)

# Dash app
app = Dash(__name__)
app.title = "SIT225 Week 8 — Smartphone Accelerometer (Live)"
app.layout = html.Div(
    style={"fontFamily":"Arial","maxWidth":"1000px","margin":"20px auto"},
    children=[
        html.H2("SIT225 Week 8 — Smartphone Accelerometer (Live)"),
        html.Div(f"Batch size N = {N_SAMPLES}. Press 'q' in the terminal to stop capture."),
        dcc.Graph(id="accel-graph", style={"height":"500px"}),
        html.Pre(id="stats-div", style={"fontFamily":"monospace","whiteSpace":"pre-wrap"}),
        html.Div(id="status", style={"color":"#555"}),
        dcc.Interval(id="tick", interval=1000, n_intervals=0),
    ]
)

@app.callback(
    [Output("accel-graph","figure"),
     Output("stats-div","children"),
     Output("status","children")],
    Input("tick","n_intervals"),
    prevent_initial_call=False
)
def tick(_):
    global _last_fig
    to_save = None
    with lock:
        n_ingest = len(ingest)
        n_hist = len(history)
        if n_ingest >= N_SAMPLES:
            to_save = [ingest.popleft() for _ in range(N_SAMPLES)]
        live_rows = list(history)[-max(PLOT_WINDOW, N_SAMPLES):] if n_hist else []
    if live_rows:
        live_df = pd.DataFrame(live_rows, columns=["timestamp","x","y","z"])
        live_df["timestamp"] = [t.strftime("%H:%M:%S") for t in live_df["timestamp"]]
        fig_live = make_fig(live_df)
        _last_fig = fig_live
        stats_text = summarize(live_df.drop(columns=["timestamp"]))
    else:
        fig_live = _last_fig if _last_fig is not None else go.Figure()
        stats_text = "Waiting for data…"
    status = f"Ingest: {n_ingest} | History window: {len(live_rows)}"
    if to_save:
        df = pd.DataFrame(to_save, columns=["timestamp","x","y","z"])
        base = ts_file()
        csv_path = os.path.join(OUT_DATA_DIR, f"accel_batch_{base}.csv")
        df_out = df.copy()
        df_out["timestamp"] = [t.strftime("%Y-%m-%d %H:%M:%S") for t in df_out["timestamp"]]
        df_out.to_csv(csv_path, index=False)
        status = f"Saved: {os.path.basename(csv_path)} | Ingest remaining: {len(ingest)}"
    return fig_live, stats_text, status

if __name__ == "__main__":
    print("Saving CSV to:", os.path.abspath(OUT_DATA_DIR))
    t_cloud = threading.Thread(target=cloud_loop, daemon=True); t_cloud.start()
    app.run(host="127.0.0.1", port=8052, debug=False)


